In [1]:
import json
from eval import main
import numpy as np
import nltk
import random
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
#import os
from sklearn.datasets import load_files
import autokeras as ak


In [2]:
#!pip install autokeras
#!pip install --upgrade tensorflow==2.12 protobuf==4.21 tensorboard==2.12 tensorflow-estimator==2.12 keras ==2.12
#!pip show tensorflow-intel 




In [3]:
def label_t(label):
    if label == 'SUPPORTS':
        t = 0
    elif label == 'REFUTES':
        t = 1
    elif label == 'NOT_ENOUGH_INFO':
        t = 2
    elif label == 'DISPUTED':
        t = 3
    else:
        return None
    return t

    

In [4]:
with open('dataset/evidence.json') as json_file:
    evidence = json.load(json_file)

with open('dataset/dev-claims.json') as json_file:
    dev_claim = json.load(json_file)

with open('dataset/dev-claims-baseline.json') as json_file:
    dev_claims_base = json.load(json_file)

with open('dataset/train-claims.json') as json_file:
    train = json.load(json_file)



In [5]:
with open('dataset/test-claims-unlabelled.json') as json_file:
    test = json.load(json_file)

In [6]:
claims_test = []
amount = []
for key in test.keys():
    test[key]['claim_text'] = test[key]['claim_text'].lower()
    sentence = test[key]['claim_text'].split()
    word = [i for i in (sentence) if i not in set(stopwords.words('english'))] 
    #print(word)
    amount += word
    word = ' '.join(word)
    claims_test.append(word)

word_amount = list(set(amount))
#print(len(word_amount))
vectorizer = CountVectorizer(max_features = len(amount))
bow_claim = vectorizer.fit_transform(claims_test).toarray()

In [11]:
responds = []
counter = 0
w_claim = []
word_amount = []
label = []
for key in train.keys():
    
    rep = label_t(train[key]['claim_label'])
    if rep == None:
        print("error response")
        break
    #else:
        #responds.append(rep)
        """
        counter +=1
        if rep in responds.keys():
            responds[rep] +=1
        else:
            responds[rep] =1
        """
    
    train[key]['claim_text'] = train[key]['claim_text'].lower()
    sentence = train[key]['claim_text'].split()
    word = [i for i in (sentence) if i not in set(stopwords.words('english'))] 
    #print(word)
    word_amount += word
    word = ' '.join(word)
    w_claim.append(word)

    sentence = train[key]['claim_label'].split()
    word = [i for i in (sentence) if i not in set(stopwords.words('english'))] 
    #print(word)
    #word_amount += word
    word = ' '.join(word)
    responds.append(word)



word_amount = list(set(word_amount))
#print(len(word_amount))
vectorizer = CountVectorizer(max_features = len(word_amount))
bow_claim = vectorizer.fit_transform(w_claim).toarray()

vectorizer = CountVectorizer()
responds = vectorizer.fit_transform(responds).toarray()


In [13]:
#code  partly from autokeras official website
sentence = []
for key in train.keys():
    sentence.append(train[key]['claim_text'])



x_train = np.array(sentence)
y_train = np.array(responds)

#print(y_train)


print(x_train.shape)  
print(y_train.shape)  
clf = ak.TextClassifier(overwrite=True, max_trials=1)  
clf.fit(x_train, y_train, epochs=2)


(1228,)
(1228, 4)

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
vanilla           |vanilla           |text_block_1/block_type
0                 |0                 |classification_head_1/dropout
5000              |5000              |text_block_1/max_tokens
False             |False             |text_block_1/conv_block_1/separable
512               |512               |text_block_1/text_to_int_sequence_1/output_sequence_length
none              |none              |text_block_1/embedding_1/pretraining
64                |64                |text_block_1/embedding_1/embedding_dim
0.25              |0.25              |text_block_1/embedding_1/dropout
5                 |5                 |text_block_1/conv_block_1/kernel_size
1                 |1                 |text_block_1/conv_block_1/num_blocks
1                 |1                 |text_block_1/conv_block_1/num_layers
False             |False             |text_block_1/conv_block_1/max_pooling
0             

: 

: 

In [11]:
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

print("Done loading BERT model.")

c:\Users\a1360\anaconda3\envs\sklearn-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exact

Done loading BERT model.


In [ ]:
#should be format of dict of dict
predict = {}



file_path = 'test-claims-predictions.json'
with open(file_path, 'w') as json_file:
    json.dump(predict, json_file)





In [2]:
print(len(['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']
 ['SUPPORTS']
 ['SUPPORTS']
 ['NOT_ENOUGH_INFO']]))

<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
C:\Users\a1360\AppData\Local\Temp\ipykernel_21616\3312600585.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  print(len([['SUPPORTS']
C:\Users\a1360\AppData\Local\Temp\ipykernel_21616\3312600585.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  print(len([['SUPPORTS']
C:\Users\a1360\AppData\Local\Temp\ipykernel_21616\3312600585.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  print(len([['SUPPORTS']
C:\Users\a1360\AppData\Local\Temp\ipykernel_21616\3312600585.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  print(len([['SUPPORTS']
C:\Users\a1360\AppData\Local\Temp\ipykernel_21616\33126005

TypeError: list indices must be integers or slices, not str